## <center><u>Building an Efficient Investment Portfolio:</u></center>
#### Demonstration of the Markowitz Model and Financial Risk Assessment Using Historical Stock Data
#### Connor Roberts | EMSE 6450 | May 10, 2017

## Intro

In this project, we will explore the Markowitz model, the basis of modern portfolio theory established by Harry Markowitz <a href="https://www.math.ust.hk/~maykwok/courses/ma362/07F/markowitz_JF.pdf">in 1952</a> for which he won the Nobel Prize in economics. This model addresses the trade-off between risk and expected returns and seeks to mathematically determine the least-risk portfolio for any desired return. 

Using the tools Markowitz gave us, we no longer have to sift through all feasible portfolios, hoping to arrive upon the perfect one. Instead, we can numerically (and sometimes analytically) establish an <i>efficient frontier</i> consisting only of efficient (i.e. minimum risk for maximum return) portfolios, then choose from these the portfolio that fits our personal appetite for risk. 

In the sections that follow, we will first accomplish this using the <b>two-fund theorem</b> and a set of risky stocks. We will then move on to include a risk-free treasury note, and will use the <b>one-fund theorem</b> to update our model. Finally, we will use these models to inform our investment decisions, and will look into how we might further try to assess and manage risk.

## Initial Calculations

To get started, we must obtain historical stock quotes from <a href="http://finance.yahoo.com"> Yahoo Finance </a>. The first step is to access a CSV file called `dow.csv` and extract the ticker symbols of the 30 stocks used in the Dow Jones Industrial Average. We'll use these as our available stocks going forward. The Dow is actually an <a href="https://www.washingtonpost.com/news/wonk/wp/2013/09/10/the-dow-jones-industrial-average-is-ridiculous/?utm_term=.5d45156d4c28"> utterly terrible </a> financial indicator, but is convenient for our purposes because the companies included are well-established and 30 is a manageable number of stocks to consider. This analysis could actually be done using a vast number of stocks (say, the S&P 500), but for the sake of simplicity and computing time, we'll consider only those in the Dow.

The historical period in question will be a nice, even 17 years, extending from Y2K to New Years Day 2017. The analysis gets stronger as the period in question is lengthened and more data is considered, so it can be tempting to download data going back 50 years or more. We can't do this, though, because many of the stocks we're considering weren't around that long ago. One might think the limiting factor here is Cisco, the Dow's youngest constituent stock, which went public in 1990. Interestingly though, Goldman Sachs - the oldest investment bank on Wall Street - only <a href="http://money.cnn.com/1999/05/03/markets/goldman/">became a publicly traded company</a> in 1999. So that is why we must settle for a measly 17 years of data. However, 17 years is practically an eon on the stock market (long enough to see two recessions and the near-doubling of the Dow) so we should be fine.

Let's get started by importing some important Python libraries that we'll need, accessing those stock ticker symbols, and grabbing the historical prices from the web.

In [22]:
#Import libraries
import numpy as np 
import csv 
import pandas as pd
pd.set_option('display.max_columns', 30) #enable all stocks to be displayed
pd.set_option('display.max_rows', 20) #abbreviate displayed data frames to save space
import pandas_datareader.data as web

#Open CSV file containing stock data and produce array of ticker symbols
with open('dow.csv','rb') as csvfile:
    stocks = csv.reader(csvfile)
    tickers = [] #list(SP100)
    for row in stocks:
        row = "".join(row)
        tickers.append(row)
              
data = [] #initiate container for stock data

#specify period over which prices are to be analyzed
startdate = '01/01/2000'
enddate = '01/01/2017'

#gather historical price data for each stock
for ticker in tickers:
    x = web.get_data_yahoo(ticker, startdate, enddate, interval ='m')['Adj Close'] #monthly adjusted close price over specified period
    data.append(x)
    
prices = pd.DataFrame(data) #convert stock data to data frame
prices = pd.DataFrame.transpose(prices) #switch date index from columns to rows
prices.columns = tickers #index columns by ticker symbols

In [23]:
prices #show prices

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2000-01-03,30.883728,38.091259,3.346637,30.483513,13.326694,23.092350,46.051998,18.292389,31.272015,26.831516,26.305365,76.824257,40.721947,84.102448,33.925117,27.846813,32.257607,24.381311,40.245003,33.028435,4.731551,20.270834,32.065639,19.360775,5.904947,18.779675,25.539255,25.539255,40.166580,28.913923
2000-02-01,29.278503,31.025015,3.697429,25.389725,11.010715,20.798775,55.593594,15.485831,26.941137,24.519764,25.986362,77.557915,41.351204,77.065681,38.758694,23.380610,31.832830,20.711884,31.511389,30.160065,2.957220,18.040838,27.881441,14.350223,5.696046,18.139355,20.178846,20.178846,35.764763,27.072586
2000-03-01,29.403006,34.435310,4.380869,25.991173,12.384602,25.741747,65.030052,15.004467,28.241514,25.394743,30.640678,88.248329,46.419727,88.784920,45.254204,22.812332,34.856201,24.381311,31.949816,35.854622,4.128115,20.532860,18.005938,22.084858,6.646646,22.501703,25.204229,25.204229,41.502598,32.845417
2000-04-03,28.759748,34.624931,4.001853,27.279993,12.485415,23.705381,58.314114,15.104363,25.307331,25.293001,30.960621,78.286972,40.662239,83.628456,43.496342,26.790283,28.984854,24.829798,35.742653,23.537506,4.525300,23.656664,19.046122,23.055620,7.433933,22.145594,24.892014,24.892014,40.676216,34.736519
2000-05-01,28.653776,37.520100,2.709567,26.948862,12.109468,25.923052,47.891975,17.062336,26.325880,27.271585,31.083675,61.758556,35.264599,80.584808,42.778065,29.168865,30.014652,23.362026,38.527325,21.112045,4.466698,25.039278,21.197777,24.269073,8.311791,21.587631,21.936087,21.936087,42.328976,33.591904
2000-06-01,27.734850,36.217316,3.378894,28.846062,10.724398,23.784706,53.464478,18.420496,23.505249,25.696259,31.305174,79.572472,35.970585,82.274414,45.865803,33.201992,27.766689,21.486542,39.559883,26.996422,4.154039,27.008659,18.089832,22.256243,9.558908,21.029524,21.080423,21.080423,42.375202,30.904552
2000-07-03,30.094818,39.447670,3.278092,33.675297,10.887394,22.150253,55.041603,19.663229,24.344723,26.248646,30.610575,82.950760,37.276150,84.292557,45.801476,30.329916,30.237268,20.671112,36.994591,23.558596,4.564878,24.406263,18.279356,28.982031,9.119980,20.850929,19.559992,19.559992,40.628719,30.705490
2000-08-01,31.269922,41.067745,3.931291,37.113155,11.751385,23.887396,57.722710,16.882107,24.298035,26.870153,34.719128,108.849892,34.647717,99.246178,51.392799,30.067545,33.917336,19.498919,36.256218,23.558596,4.129582,24.326498,19.827681,30.989574,10.536047,22.372200,18.206734,18.206734,35.021591,31.016506
2000-09-01,30.634340,42.274681,1.661222,44.626617,9.908527,24.094965,46.472565,17.735222,22.124502,29.328306,34.317635,95.874062,38.162052,84.668793,28.527721,30.716419,28.036814,19.692598,38.614952,20.352772,4.186628,25.330896,21.486259,33.034870,11.008073,24.858000,20.266033,20.266033,35.428635,30.456659


Now that all of the price data has been acquired, we can calculate the monthly returns as follows:

$$R_k = \frac{P_{k+1} - P_k}{P_k}$$

where $P_k$ is the closing price in month $k$. 

(This is as good a time as any to mention that, as you see above and below, we'll occasionally make a point of displaying tables full of relevant data and outputs. They are included here to benefit you as the reader, in case you want to scroll through and get a better look at these values)

In [554]:
months = len(prices) - 1. #number of months captured in data

returnarray = (np.asarray(prices[:][1:]) - np.asarray(prices[:][0:-1]))/np.asarray(prices[:][0:-1]) #convert price components to arrays and calculate returns        
returnarray[np.isnan(returnarray)] = 0 #sometimes no data is available on a given day, yielding a NaN. We replace those with zeros and trust that they're infrequent enough not to harm the analysis.
returns = pd.DataFrame(returnarray, index = range(int(months)), columns = tickers) #convert returns back to dataframe

returns #show returns

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
0,-0.051976,-0.185508,0.104819,-0.167100,-0.173785,-0.099322,0.207192,-0.153428,-0.138491,-0.086158,-0.012127,0.009550,0.015453,-0.083669,0.142478,-0.160385,-0.013168,-0.150502,-0.217011,-0.086845,-0.375000,-0.110010,-0.130489,-0.258799,-0.035377,-0.034096,-0.209889,-0.209889,-0.109589,-0.063683
1,0.004252,0.109921,0.184842,0.023689,0.124777,0.237657,0.169740,-0.031084,0.048267,0.035685,0.179106,0.137838,0.122573,0.152068,0.167588,-0.024306,0.094977,0.177165,0.013913,0.188811,0.395945,0.138132,-0.354196,0.538991,0.166888,0.240491,0.249042,0.249042,0.160433,0.213235
2,-0.021877,0.005507,-0.086516,0.049587,0.008140,-0.079108,-0.103274,0.006658,-0.103896,-0.004006,0.010442,-0.112879,-0.124031,-0.058078,-0.038844,0.174377,-0.168445,0.018395,0.118712,-0.343529,0.096215,0.152137,0.057769,0.043956,0.118449,-0.015826,-0.012387,-0.012387,-0.019912,0.057576
3,-0.003685,0.083615,-0.322922,-0.012138,-0.030111,0.093551,-0.178724,0.129630,0.040247,0.078227,0.003975,-0.211126,-0.132743,-0.036395,-0.016514,0.088785,0.035529,-0.059113,0.077909,-0.103047,-0.012950,0.058445,0.112971,0.052632,0.118088,-0.025195,-0.118750,-0.118750,0.040632,-0.032951
4,-0.032070,-0.034722,0.247024,0.070400,-0.114379,-0.082488,0.116356,0.079600,-0.107143,-0.057764,0.007126,0.288445,0.020020,0.020967,0.072180,0.138268,-0.074896,-0.080279,0.026801,0.278721,-0.069998,0.078652,-0.146617,-0.082938,0.150042,-0.025853,-0.039007,-0.039007,0.001092,-0.080000
5,0.085090,0.089194,-0.029833,0.167414,0.015199,-0.068719,0.029499,0.067465,0.035714,0.021497,-0.022188,0.042455,0.036295,0.024529,-0.001403,-0.086503,0.088976,-0.037951,-0.064846,-0.127344,0.098901,-0.096354,0.010477,0.302198,-0.045918,-0.008493,-0.072125,-0.072125,-0.041215,-0.006441
6,0.039047,0.041069,0.199262,0.102088,0.079357,0.078425,0.048711,-0.141438,-0.001918,0.023678,0.134220,0.312223,-0.070512,0.177401,0.122077,-0.008651,0.121706,-0.056707,-0.019959,0.000000,-0.095358,-0.003268,0.084703,0.069269,0.155271,0.072959,-0.069185,-0.069185,-0.138009,0.010129
7,-0.020326,0.029389,-0.577436,0.202447,-0.156820,0.008689,-0.194900,0.050534,-0.089453,0.091483,-0.011564,-0.119208,0.101430,-0.146881,-0.444908,0.021581,-0.173378,0.009933,0.065057,-0.136079,0.013814,0.041288,0.083650,0.065999,0.044801,0.111111,0.113106,0.113106,0.011623,-0.018050
8,0.060357,-0.011059,-0.240291,0.051357,0.143799,-0.036657,-0.024887,0.095238,0.110092,0.000878,-0.051892,-0.123859,-0.187721,-0.125416,0.082707,-0.019295,-0.008095,0.026915,0.208228,0.141969,-0.003120,-0.038607,0.071346,0.017370,0.107595,0.006306,0.202916,0.202916,-0.057143,-0.063725
9,0.040010,-0.084375,-0.156549,0.020625,0.121212,0.004707,-0.111369,0.040193,-0.059784,-0.008504,-0.095781,-0.177207,-0.087721,-0.049554,-0.153804,0.089182,-0.189560,0.034956,0.030577,-0.166969,0.067293,0.028211,0.048119,-0.021951,0.072571,0.017726,-0.028108,-0.028108,0.150138,-0.191972


With the returns calculated, we are clear to do some more interesting work. First, we evaluate the mean return and standard deviation for each stock. 

In [25]:
means = np.matrix(np.mean(returns)) #matrix of mean returns (matrix used instead of array to allow for transpose function later)
stddevs = np.matrix(np.std(returns)) #matrix of standard deviations of returns

#show means and stddevs
pd.DataFrame(np.vstack((means,stddevs)), index = ['Means', 'Std. Devs'], columns = tickers)

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
Means,0.010262,0.007387,0.025581,0.011357,0.013928,0.009787,0.003263,0.005302,0.007077,0.007112,0.003751,0.010053,0.008573,0.005846,0.005640,0.008086,0.009003,0.009604,0.004421,0.006901,0.014829,0.003847,0.006146,0.011705,0.019071,0.010724,0.005783,0.005783,0.004018,0.008888
Std. Devs,0.057029,0.096815,0.121675,0.080519,0.093588,0.059936,0.102461,0.050817,0.076695,0.049518,0.075291,0.094654,0.073496,0.072546,0.100714,0.046804,0.090861,0.057610,0.071239,0.087610,0.077619,0.056805,0.052084,0.074507,0.072244,0.062784,0.068280,0.068280,0.053094,0.070686


Next, the covariance matrix of the stocks, denoted $\Sigma$, is calculated using 

$$\Sigma = \frac{X^TX}{(n-1)}$$ 

where $X$ denotes the joint return data, and $(n-1)$ is the number of months being observed.

In [27]:
cov = np.matrix(returns.transpose())*np.matrix(returns)/months #covariance of stock returns
cov_inv = np.linalg.inv(cov) #inverse of covariance of stock returns

#show covariance matrix
pd.DataFrame(cov, index = tickers, columns = tickers) 

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
MMM,0.003358,0.002962,0.001599,0.002018,0.003260,0.001423,0.001972,0.000946,0.002786,0.000959,0.002173,0.001905,0.001814,0.001230,0.001894,0.001047,0.001998,0.000961,0.000827,0.000875,0.001621,0.000936,0.000972,0.001791,0.001146,0.002038,0.000909,0.000909,0.000523,0.001974
AXP,0.002962,0.009428,0.003358,0.004009,0.005098,0.001856,0.004021,0.001266,0.004555,0.001256,0.004203,0.003861,0.003034,0.002403,0.003444,0.001311,0.004731,0.001722,0.001326,0.002470,0.002885,0.001418,0.001220,0.002636,0.002223,0.003350,0.001528,0.001528,0.000836,0.003929
AAPL,0.001599,0.003358,0.015459,0.001645,0.003123,0.002362,0.004741,0.000226,0.002463,0.001206,0.002471,0.005398,0.002806,0.004341,0.007360,0.000182,0.003409,0.001262,-0.000307,0.005106,0.001964,0.000166,-0.000636,0.001524,0.001717,0.001962,0.001340,0.001340,0.000276,0.003267
BA,0.002018,0.004009,0.001645,0.006612,0.002891,0.001866,0.002566,0.001364,0.003034,0.001547,0.002991,0.002355,0.001852,0.000837,0.001713,0.001068,0.002246,0.001681,0.001829,0.001228,0.001772,0.001525,0.000992,0.002207,0.001975,0.003344,0.001143,0.001143,0.000388,0.002994
CAT,0.003260,0.005098,0.003123,0.002891,0.008953,0.002967,0.003654,0.001205,0.004565,0.002036,0.003774,0.004105,0.002577,0.002389,0.003573,0.001152,0.003932,0.002151,0.001516,0.002396,0.003045,0.001683,0.001173,0.002744,0.001764,0.003399,0.002122,0.002122,0.001168,0.003389
CVX,0.001423,0.001856,0.002362,0.001866,0.002967,0.003688,0.001442,0.000981,0.002053,0.002317,0.001816,0.001823,0.001267,0.001450,0.001249,0.000654,0.001799,0.001545,0.000759,0.001683,0.001267,0.001126,0.000352,0.002035,0.001046,0.001772,0.001426,0.001426,0.000371,0.001805
CSCO,0.001972,0.004021,0.004741,0.002566,0.003654,0.001442,0.010509,0.000917,0.003268,0.000753,0.003348,0.004662,0.002977,0.003878,0.006797,0.000464,0.004412,0.001656,0.000891,0.004256,0.001786,0.001347,0.000317,0.001846,0.000757,0.002852,0.001277,0.001277,0.000828,0.003306
KO,0.000946,0.001266,0.000226,0.001364,0.001205,0.000981,0.000917,0.002610,0.001176,0.000855,0.001129,0.000670,0.000645,0.000376,0.000606,0.001096,0.001002,0.001285,0.001287,0.000933,0.001253,0.000968,0.001071,0.001291,0.000416,0.000608,0.000971,0.000971,0.000618,0.000998
DD,0.002786,0.004555,0.002463,0.003034,0.004565,0.002053,0.003268,0.001176,0.005932,0.001468,0.003593,0.003320,0.002315,0.002237,0.003128,0.001222,0.003803,0.001652,0.001753,0.002398,0.002475,0.001545,0.000867,0.002422,0.001657,0.002942,0.001539,0.001539,0.000714,0.003303
XOM,0.000959,0.001256,0.001206,0.001547,0.002036,0.002317,0.000753,0.000855,0.001468,0.002503,0.001348,0.001102,0.000620,0.000968,0.000627,0.000661,0.001091,0.001081,0.001021,0.000959,0.000686,0.000954,0.000629,0.001467,0.000741,0.001215,0.001319,0.001319,0.000239,0.001163


We could also calculate the mean-centered and standardized returns and covariances (as shown below). This is how we build a correlation matrix, but we won't spend time delving into it this, as it doesn't contribute to the Markowitz model being built here.

In [28]:
returnsmc = np.matrix(returns) - means #mean-centered returns
returnsstd = returnsmc/stddevs #standardized returns

cov_mc = np.matrix(returnsmc.transpose())*np.matrix(returnsmc)/months #mean-centered covariance
cov_std = np.matrix(returnsstd.transpose())*np.matrix(returnsstd)/months #standardized covariance (i.e. correlation)

## Building an Efficient Frontier

### Markowitz Model

With the means and covariance calculated, the stage is set and we can finally get into the really interesting stuff. We're going to take a look at exactly what the Markowitz model is and how we can use it. In the Markowitz model, we pick some desired, possibly arbitrary rate of return, $\alpha$, then find the feasible portfolio with the least variance (i.e. least risk) that achieves this return. It boils down to a minimization of the objective, variance, subject to portfolio return being equal to $\bar{r}$ and the weights of each stock (i.e. what percentage of funds are used to buy shares of that stock) summing to 1. The problem can be described thusly

$$\text{minimize} \quad \frac{1}{2}\sum_{i,j = 1}^nw_iw_j\sigma_{ij}\\
\text{subject to} \quad \sum_{i = 1}^nw_i\bar{r}_i = \alpha \\
 \quad \quad \quad \quad \sum_{i = 1}^nw_i = 1$$

where $\bar{r}_i$ and $w_i$ is the return and weight, respectively, of each stock $i$. The $\frac{1}{2}$ coefficient in the objective function is arbitrary, and is included only for convenience in the next step.

So what is that next step? Finding an analytical solution of course! We can accomplish this by converting our optimization problem into a <i>Lagrangian</i>. First, each constraint equation is rearranged so that the right-hand side is zero. They are then each multiplied by a device called a <i>Lagrange multiplier</i>, $\lambda_1$ and $\lambda_2$ in our case, and subtracted from the objective function. This yields the Lagrangian

$$ L = \frac{1}{2}\sum_{i,j = 1}^nw_iw_j\sigma_{ij} - \lambda_1(\sum_{i = 1}^nw_i\bar{r}_i - \alpha) - \lambda_2(\sum_{i = 1}^nw_i-1)$$

Expressed in matrix-vector, this is 

$$ L = \frac{1}{2}w^T\Sigma w - \lambda_1(w^T\bar{r} - \alpha) - \lambda_2(w^T1_N - 1) $$

where $w$ denotes the set of all weights, $w_i$, and $1_N$ represents a vector of length $N$ where each element is 1. We seek to choose weights that minimize this, so we take the partial derivatives with respect to each weight and set them equal to zero. This creates a system of equations consisting of $N$ equations (where $N$ is the number of stocks) plus two more equations corresponding to our constraints:

$$ \frac{\partial L}{\partial w} = 0_N = \Sigma w - \lambda_1\bar{r} - \lambda_21_N \\
  \frac{\partial L}{\partial \lambda_1} = 0 = \alpha - w^T\bar{r} \\
  \frac{\partial L}{\partial \lambda_2} = 0 = 1 - w^T1_N$$

### Two-fund Theorem

Let's suppose we find two solutions: $w^1$,$\lambda_1^1$,$\lambda_2^1$, and $w^2$,$\lambda_1^2$,$\lambda_2^2$, with each solution having corresponding returns and standard deviations. As it turns out, we can construct any feasible, efficient portfolio as a combination of these two solutions. This takes the form of 

$$  w_i = \beta w_{i}^1 + (1-\beta)w_{2}^2\\
\bar{r}(\beta) = \beta\bar{r}^1 + (1-\beta)\bar{r}^2 $$


where $\beta$ can take any value. In fact, by sweeping $\beta$ from $-\infty$ to $\infty$, we can cover the entire minimum-variance feasible set. This is fantastic! We only have to go through the hard work of finding a solution twice, then we can simply and painlessly construct all other solutions. For the sake of convenience, we will determine our two basis portfolios using $\lambda_1^1 = 0$, $\lambda_2^1 = 1$ and $\lambda_1^2 = 1$, $\lambda_2^2 = 0$.

In [307]:
#Point 1 where lambda = 0, mu = 1
v1 = np.linalg.solve(cov, np.matrix(np.ones(len(tickers))).transpose())#solution vector
w1 = v1/sum(v1) #normalize solution to find weights

#Point 2 where lambda = 1, mu = 0
v2 = np.linalg.solve(cov, means.transpose()) #solution vector
w2 = v2/sum(v2) #normalize solution to find weights

With $w^1$ and $w^2$ established, we can now sweep $\beta$ to produce the minimum-variance set (or at least a portion of it).


In [317]:
#construct beta portfolios

lb = -1 #lower bound
ub = 1 #upper bound
steps = 1000 #determine analysis resolution
beta = np.linspace(lb, ub, steps) #constrcut list of beta values to consider

w3 = np.array(beta)*np.array(w1) + np.array(1 - beta)*np.array(w2) #matrix where rows are stocks and each column is a set of portfolio weights corresponding to a differnt beta 
r3 = means*w3 #use matrix of weights to calculate corresponding sets of portfolio returns
var3 = np.diag((w3.transpose()*(cov*w3))) #use matrix of weights to calculate corresponding sets of variances
std3 = np.sqrt(var3) #return standard deviation for each considered beta scenario

Let's go ahead and plot the efficient frontier of portfolios consisting of the 30 Dow stocks. Traditionally, Python users might use `matplotlib`, but here we use `plotly` for its powerful, built-in user interface. With Plotly, you can hover over any stock point, or any point in the efficient frontier, and it will display the corresponding rate of return and standard deviation. You can also zoom in to get a better look at the data, and can click on the legend to include/exclude datasets. Go ahead and give it a try!

In [352]:
#import plotly libraries
import plotly.plotly as py
import plotly.graph_objs as go 

# Create a trace
trace3 = go.Scatter(
    x = std3.transpose(),
    y = mean3.transpose(),
    name = 'Eff. Frontier'
)

stockpoints = go.Scatter( 
    x = stddevs.transpose(),
    y = means.transpose(),
    mode = 'markers+text',
    name = 'stocks',
    text = tickers,
    textposition='top'
)


#points of interest 
misc = go.Scatter(
    x = [float(min_std)],
    y = [float(min_r)],
    mode = 'markers+text',
    text = ['Min. Var.'],
    name='POI',
    textposition='bottom'
)


data = [trace3, stockpoints, misc]


layout = dict(
    xaxis=dict(
        title='Standard Deviation Monthly Return'),
    yaxis=dict(
        title='Average Monthly Return')
    )


fig = dict(data=data, layout=layout)
py.iplot(fig)

One of the first things that stands out is how far to the left the efficient frontier lies in comparison to the stocks. This is evidence of the well-known principle that by investing in several assets, one can reduce one's exposure to risk (assuming the asset returns are non perfectly correlated, that is). 

This model allows us to pick our desired portfolio return and use that to determine how much we should invest in each stock. To explore this, let's assume we are highly risk averse and want to construct a portfolio with the least possible variance. How much should we invest in each stock? To figure this out, we can pick the minimum-variance point from the efficient frontier and see which set of weights corresponds to this point:

In [323]:
min_loc = np.argmin(std3) #index containing lowest variance

#minumum variance coordinates
min_std = std3[min_loc]
min_r = r3[:,min_loc]

pd.DataFrame(np.matrix(w3[:,min_loc]*100), index = ['Percent of funds invested in each stock'], columns = tickers)

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
Percent of funds invested in each stock,12.699933,-2.88124,0.79388,-3.720896,-11.055246,-0.054036,-0.117888,9.180347,-3.808052,22.15316,-3.477778,3.484604,1.753254,4.96122,0.336693,3.297337,-3.169858,2.272827,0.20431,3.987782,1.029766,2.490847,21.270522,-3.772434,6.721143,7.300599,7.54065,-7.453544,22.802974,5.229124


Note that some of the weights here are negative! How are we supposed to invest -11% of our funds in Caterpillar? That's where the idea of <i>short selling</i> (also known as shorting) comes in.

### Short Selling


Short selling a stock involves a somewhat complicated process in which one borrows shares of the stock, sells them on the market, then buys those shares at a later time to return to the lender. Stock brokers and trading websites make short selling stocks easy and straightforward, concealing the underlying complexity of the maneuver so that it boils down to a simple interaction where the investor benefits from the asset <i>depreciating</i> in value (the exact opposite of a regular stock purchase). 

Short selling is risky for a number of reasons. Firstly, the investor may be expected to continue contributing capital over the life of the position in order to keep it "alive" should the asset value increase. This differs from a typical stock purchase where the amount of capital invested is fixed and contributed up front. Secondly, in a typical stock purchase, the investor can only lose up to the amount of capital invested, as an asset's value can only drop to zero and no further. Because an asset's value can increase with no definite ceiling, a short position could theoretically yield infinite losses. <a href="http://www.marketwatch.com/story/help-my-short-position-got-crushed-and-now-i-owe-e-trade-10644556-2015-11-19">This article</a>, about a man who <i>"went to bed Wednesday evening with some \$37,000 in his trading account...and woke up to a debt of \$106,445.56."</i> serves as a cautionary tale.

For these reasons, big and small investors alike often avoid short selling stocks. But how would this affect our model? To figure this out, we can revisit our optimization problem, with the added constraint that all weights must be greater than or equal to zero. Unlike the last model formulation, this one does not have an analytical solution and must be solved numerically. Because the objective is quadratic and the constraints are linear, we will use a <i>quadratic programming</i> method, performed using tools from the `cvxpy` Python library. To suit the `cvxpy` syntax, we will reformulate the Markowitz problem as has been done in <a href="http://nbviewer.jupyter.org/github/cvxgrp/cvx_short_course/blob/master/applications/portfolio_optimization.ipynb">this example</a> so that looks like

$$\text{maximize}\quad \bar{r}w - \gamma w^T\Sigma w \\
\text{subject to}\quad 1^Tw = 1, \quad w\in {\rm I\!R_+^n}$$

where $\gamma$ is a <i>risk aversion </i> parameter. Note that this is the exact same model we have been using all along, just with the $\frac{1}{2}$ term replaced by the risk aversion parameter, and the objective function reformulated as a difference between portfolio return and risk, known as the <i>risk-adjusted return</i>, which we seek to maximize.

In [223]:
#import cvxpy and scipy libraries
from cvxpy import *
import scipy as sp

w4 = Variable(len(tickers)) #establish weights as variables
gamma = Parameter(sign = 'positive') #gamma must be positive (unless we suddenly are risk lovers)
ret = means*w4 #return associated with given set of weights
risk = quad_form(w4, cov) #wT*Sigma*w
prob = Problem(Maximize(ret - gamma*risk), 
              [sum_entries(w4) == 1,
              w4 >= 0 ]) #formulate problem

#initialize arrays
r4 = np.zeros(steps)
std4 = np.zeros(steps)
w4s = np.zeros([len(tickers),steps])

gamma_vals = np.logspace(-2,3, num= steps) #establish logarithmic range of considered gamma values

#solve problem for each considered gamma value and save outputs
for i in range(steps):
    gamma.value = gamma_vals[i]
    prob.solve(solver=ECOS)
    w4s[:,[i]] = w4.value
    r4[i] = ret.value
    std4[i] = sqrt(risk).value    

With the return and standard deviation values calculated for the restricted portfolio, let's go ahead and graph the efficient frontier without shorting.

In [373]:
# Create a trace
trace1 = go.Scatter(
    x = std3.transpose(),
    y = mean3.transpose(),
    name = 'Eff. Frontier 1'
)

trace2 = go.Scatter(
    x = std4.transpose(),
    y = r4.transpose(),
    name = 'Eff. Frontier 2'
)

stockpoints = go.Scatter( 
    x = stddevs.transpose(),
    y = means.transpose(),
    mode = 'markers+text',
    name = 'stocks',
    text = tickers,
    textposition='top'
)

#points of interest 
misc = go.Scatter(
    x = [float(min_std),float(std4[-1])],
    y = [float(min_r),float(mean4[-1])],
    mode = 'markers+text',
    text = ['MV1','MV2'],
    name='POI',
    textposition='bottom'
)



data = [trace1, trace2, stockpoints, misc]


layout = dict(
    xaxis=dict(
        title='Standard Deviation Monthly Return'),
    yaxis=dict(
        title='Average Monthly Return')
    )


fig = dict(data=data, layout=layout)
py.iplot(fig)

Looking at this plot, we can make a few observations about our restricted portfolio. Firstly, its minimum-variance point has roughly the same return as the unrestricted portfolio, but is slightly riskier. Secondly, it has a finite "end" at the right. Even if we were to widen the range of risk values the simulation may consider, that point will always be the highest risk, highest return portfolio possible. This is due to our restriction on short selling. Finally, if we look more deeply into the weights of this specific scenario, it becomes apparent that the high-risk/high-return portfolio involves investing in only a single stock (Apple, in this case). Why then, does the curve not stop right at the point where we see the 'AAPL' data plotted? This simply is a result of imperfections in the solver used (ECOS, in this case), and it is something we must always be weary of when applying numerical solving methods. A different solver (e.g. GPLK, CVXOPT) may serve this problem better, but that is beyond the scope of this project.

## Risk-free Asset

Now we will explore how our investment decisions might be impacted by the inclusions of a risk-free asset, such as a treasury note. To do this, first we must acquire <a href="https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield">treasury note yield data</a> from the US Treasury. We will accomplish this using the `quandl` financial tool, as shown below.

In [35]:
import quandl as ql
yield_df = ql.get("USTREASURY/YIELD", collapse="monthly", start_date = startdate, end_date = enddate) #download yield data
yield_df #show yield schedule over time

,1 MO,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,,
2000-01-31,NaN,5.76,5.97,6.30,6.61,6.65,6.71,6.75,6.68,6.72,6.49
2000-02-29,NaN,5.78,6.02,6.20,6.53,6.58,6.61,6.67,6.42,6.46,6.15
2000-03-31,NaN,5.88,6.15,6.28,6.50,6.44,6.32,6.28,6.03,6.20,5.84
2000-04-30,NaN,5.82,6.12,6.24,6.68,6.64,6.56,6.49,6.23,6.31,5.97
2000-05-31,NaN,5.63,6.35,6.37,6.69,6.66,6.54,6.52,6.29,6.42,6.02
2000-06-30,NaN,5.88,6.23,6.08,6.38,6.31,6.18,6.25,6.03,6.26,5.90
2000-07-31,NaN,6.27,6.42,6.07,6.30,6.24,6.16,6.19,6.04,6.13,5.79
2000-08-31,NaN,6.31,6.38,6.22,6.18,6.09,5.98,5.98,5.73,5.96,5.67
2000-09-30,NaN,6.23,6.28,6.07,5.98,5.91,5.85,5.93,5.80,6.13,5.88


### One-fund Theorem

By allowing for the option to invest in a risk-free asset, we arrive at the one-fund theorem. This theorem asserts that there is some portfolio of risky assets, $F$, that can be weighted in combination with the risk-free asset in order to produce a new, better efficient frontier. Using $\alpha$ as the risk-free weight (not to be confused with the earlier $\alpha$ that represented an arbitrary portfolio return), we can express this as

$$  \bar{r}(\alpha) = \alpha\bar{r}_{rf} + (1-\alpha)\bar{r}_F \\
\sigma(\alpha) = \alpha\sigma_{rf} + (1-\alpha)\sigma_F$$

where $\bar{r}_{rf}$ is the risk-free rate of return and $\bar{r}_F$ is the expected return from fund $F$. By sweeping $\alpha$ from 0 (no investment in risk-free asset) to 1 (investment only in risk-free asset), we can construct a new efficient frontier.

In [427]:
yield_1yr = yield_df.iloc[-1,3] #acquire 1 year interest rate
rf = ((1+yield_1yr/100)**(1/12.)- 1) #convert this to a monthly compounding rate

#find the tangent point, which is fund F
v5 = np.linalg.solve(cov, (means - rf).transpose()) #solution vector
w5 = v5/sum(v5) #normalize solution to find weights
r5 = w5.transpose()*means.transpose() #return at tangent point
var5 = w5.transpose()*(cov*w5) #variance at tangent point
std5 = np.sqrt(var5) #standard deviation at tangent point

#Tangent line prepared for graphing
rf_x1 = np.linspace(0,float(std5),steps)
rf_y1 = np.linspace(rf,float(r5),steps)

In [428]:
# Create a trace
trace1 = go.Scatter(
    x = std3.transpose(),
    y = mean3.transpose(),
    name = 'Two-fund Eff. Frontier'
)

trace2 = go.Scatter(
    x = rf_x1,
    y = rf_y1,
    name = 'One-fund Eff. Frontier'
)

stockpoints = go.Scatter( 
    x = stddevs.transpose(),
    y = means.transpose(),
    mode = 'markers+text',
    name = 'stocks',
    text = tickers,
    textposition='top'
)

#points of interest
misc = go.Scatter(
    x = [0,float(min_std),float(std5)],
    y = [rf,float(min_r),float(r5)],
    mode = 'markers+text',
    text = ['RF','Min. Var.','F'],
    name='POI',
    textposition='right'
)

data = [trace1, trace2, stockpoints, misc]

layout = dict(
    xaxis=dict(
        title='Standard Deviation Monthly Return'),
    yaxis=dict(
        title='Average Monthly Return')
    )


fig = dict(data=data, layout=layout)
py.iplot(fig)

So there is our new and improved efficient frontier, which incorporates the risk-free asset. But how might this look if we restricted short selling?

Just as there was no nice, closed-form way to solve for the two-fund efficient frontier without short selling, the same goes for the tangent line in the one-fund theorem. We will have to use a more numerical, brute-force approach  by calculating $\frac{\Delta y}{\Delta x}$ for each small segment along our generated efficient frontier ($y$ here of course implying portfolio returns and $x$ implying standard deviation), then rearranging the classic $y=mx+b$ linear equation to yield $b = y-mx = y - \frac{\Delta y}{\Delta x}x$. Here, $b$ is used to denote the y-axis intercept, which in our case represents the zero-variance return of the tangent line. We can repeat this for every point along our efficient frontier, then parse the results to find which tangent line passes through our risk-free return ratio. Once we find that, we've got our restricted one-fund theorem efficient frontier!

In [429]:
r_int = np.zeros(steps) #initialize intercept of 'returns' axis

#for each element, solve for the intercept of the tangent line
for i in range(steps - 1):
    dydx = (mean4[i+1] - mean4[i])/(std4[i+1] - std4[i])
    r_int[i] = mean4[i] - dydx*std4[i]
    
index = np.where(np.around(r_int, 4) == round(rf,4))
r6 = mean4[index]
std6 = std4[index]

w6 = np.around(w4s[:, index],5)[:,0,0] #corresponding weights at tangent point. Values are rounded and stored in an array

#Tangent line prepared for graphing
rf_x2 = np.linspace(0,float(std6),steps)
rf_y2 = np.linspace(rf,float(r6),steps)

Let's graph it to make sure it looks right.

In [431]:
# Create a trace
trace1 = go.Scatter(
    x = std4.transpose(),
    y = mean4.transpose(),
    name = 'Two-fund Eff. Frontier'
)

trace2 = go.Scatter(
    x = rf_x2,
    y = rf_y2,
    name = 'One-fund Eff. Frontier'
)

stockpoints = go.Scatter( 
    x = stddevs.transpose(),
    y = means.transpose(),
    mode = 'markers+text',
    name = 'stocks',
    text = tickers,
    textposition='top'
)

#points of interest
misc = go.Scatter(
    x = [0,float(std4[-1]),float(std6)],
    y = [rf,float(mean4[-1]),float(r6)],
    mode = 'markers+text',
    text = ['RF','Min. Var.','F'],
    name='POI',
    textposition='right'
)

data = [trace1, trace2, stockpoints, misc]

layout = dict(
    xaxis=dict(
        title='Standard Deviation Monthly Return'),
    yaxis=dict(
        title='Average Monthly Return')
    )


fig = dict(data=data, layout=layout)
py.iplot(fig)

Voila! Looks great!

## Investment Decision

So that's it! The model is all set up. Now we can use it to inform our investment decision. Let's assume we have \$1,000,000 to invest and we want to minimizes our risk while aiming for a 10% return over the course of a year. How much should we put into a risk-free asset like a treasury note? How much should we invest into the stock market and, specifically how much should we buy of each stock?

With a few more lines of Python code, we can figure all of that out. 

First let's establish the basic parameters of our investment situation.

In [444]:
investment = 1e6 #amount available for investment
rd = 10/100. #return desired
rd = ((1+rd)**(1/12.)- 1) #desired monthly compounding return

Assuming short selling is permitted, we now determine our optimal investment decisions:

In [672]:
#Short selling permitted

abs_w5 = np.abs(w5) #absolute values of weights

percentinrf = (rd - r5)/(rf - r5) #percent of funds invested in risk-free asset
percentinF =  1 - percentinrf#percent of funds invested in stock market

stdreturn = percentinF*std5 #risk exposure

F_funds = sum(abs_w5)*investment #funds required for investment in F
borrow_F = F_funds - investment #how much needs to be borrowed to fund F investment
borrow_rf = (percentinrf < 0)*np.abs((percentinrf)*F_funds) #how much needs to be borrowed to invest in risk-free assets
req_funds = F_funds + borrow_rf #total required funds
borrow = borrow_F + borrow_rf #total borrowed funds

In [673]:
#Weights of fund F
pd.DataFrame(w5.transpose()*100, columns = tickers, index = ['Percent invested in each stock'])

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
Percent invested in each stock,24.345734,-20.988607,24.980542,11.007908,12.604404,-8.406915,0.596457,-5.064131,-17.546979,12.058007,-26.648702,-9.776572,2.14088,-2.414538,-21.861467,38.290009,15.675825,10.453464,-3.403876,11.607207,18.71567,-33.680419,9.551269,7.80056,37.722489,9.231898,16.578067,-20.516782,1.803348,5.145253


In [675]:
print '%.2f'%float(percentinF*100),'% invested in stocks' 
print '%.2f'%float(percentinrf*100),'% invested in risk-free assets' 
print '$%.2f total investment required' %float(req_funds)
print '$%.2f must be borrwed' %float(borrow)
print '%.2f'%float(((1+stdreturn)**12 - 1)*100),'% is the standard deviation (risk) of this investment'

32.11 % invested in stocks
67.89 % invested in risk-free assets
$4406179.79 total investment required
$3406179.79 must be borrwed
23.94 % is the standard deviation (risk) of this investment


So this is telling us that in order to achieve a 10% return on investment over one year while exposing ourselves to the least risk possible, we should first borrow \$3,406,180, so that we have \$4,406,180 in total funds. This large sum is necessary in order to fund the short positions in fund $F$, in which we invest 32.1% of our funds. The remaining 67.9% of our funds are used to purchase one-year treasury notes. This, according to the Markowitz model, is our optimal investment option.

But what if we don't want to invest in a fund that short sells? This is probably a prudent move, so how might it affect our decisions in constructing our portfolio?

In [676]:
#Short selling is restricted
abs_w6 = np.abs(w6) #absolute values of weights

percentinrf = (rd - r6)/(rf - r6) #percent of funds invested in risk-free asset
percentinF =  1 - percentinrf#percent of funds invested in stock market


stdreturn = percentinF*std6 #risk exposure
F_funds = sum(abs_w6)*investment #funds required for investment
borrow_F = F_funds - investment #how much needs to be borrowed to fund stock investment
borrow_rf = (percentinrf < 0)*np.abs((percentinrf)*F_funds) #how much needs to be borrowed to invest in risk-free assets
req_funds = F_funds + borrow_rf
borrow = borrow_F + borrow_rf #total borrowed funds

In [677]:
#Weights of fund F
pd.DataFrame((np.matrix(w6*100.)), columns = tickers, index = ['Percent invested in each stock'])

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
Percent invested in each stock,5.101,0.0,16.51,0.0,0.0,0.0,0.0,0.0,0.0,4.905,0.0,0.0,0.0,0.0,0.0,7.303,0.0,5.044,0.0,0.0,14.918,0.0,12.483,1.128,32.608,0.0,0.0,0.0,0.0,0.0


In [678]:
print '%.2f'%float(percentinF*100),'% invested in stocks' 
print '%.2f'%float(percentinrf*100),'% invested in risk-free assets' 
print '$%.2f total investment required' %float(req_funds)
print '$%.2f must be borrwed' %float(borrow)
print '%.2f'%float(((1+stdreturn)**12 - 1)*100),'% is the standard deviation (risk) of this investment'

49.13 % invested in stocks
50.87 % invested in risk-free assets
$1000000.00 total investment required
$0.00 must be borrwed
30.75 % is the standard deviation (risk) of this investment


In this case, we don't need to borrow funds because we aren't short selling any stocks. For proof that short selling is no longer in play, take a look at the portfolio makeup printed above. So, we keep our original \$1,000,000 in funds and we invest 49.1% in fund $F$ and 50.9% in treasury notes. We avoid the risk involved in short selling (which is, unfortunately, not captured quantitatively in this analysis), but we pay for it with an extra $\approx$ 7% in the standard deviation (i.e. risk) of our expected returns.

This is excellent, useful information. However, it is unusual to invest in the market for only a year. For our final portfolio calculation, let's examine what our decisions should look like if we plan to avoid short selling and invest for 10 years. We will follow the exact same steps, but we've completed much of the legwork already, so the code is pretty short.

In [679]:
yield_10yr = yield_df.iloc[-1,8] #acquire 10-year annual interest rate
rf_10 = ((1+yield_10yr/100)**(1/12.)- 1) #convert this to a monthly compounding rate

#find first location where r_int is about equal to rf_10
index_10 = np.where(np.around(r_int, 4) == round(rf_10,4))[0]
index_10 = index_10[0]

r7 = mean4[index_10] #return of fund F
std7 = std4[index_10] #standard deviation of fund F
w7 = np.around(w4s[:, index_10],8) #corresponding weights at tangent point. Values are rounded and stored in an array

abs_w7 = np.abs(w7) #absolute values of weights

percentinrf = (rd - r7)/(rf - r7) #percent of funds invested in risk-free asset
percentinF =  1 - percentinrf#percent of funds invested in stock market


stdreturn = percentinF*std7 #risk exposure
F_funds = sum(abs_w7)*investment #funds required for investment
borrow_F = F_funds - investment #how much needs to be borrowed to fund stock investment
borrow_rf = (percentinrf < 0)*np.abs((percentinrf)*F_funds) #how much needs to be borrowed to invest in risk-free assets
req_funds = F_funds + borrow_rf
borrow = borrow_F + borrow_rf #total borrowed funds

In [680]:
#Weights of fund F
pd.DataFrame((np.matrix(w7*100.)), columns = tickers, index = ['Percent invested in each stock'])

,MMM,AXP,AAPL,BA,CAT,CVX,CSCO,KO,DD,XOM,GE,GS,HD,IBM,INTC,JNJ,JPM,MCD,MRK,MSFT,NKE,PFE,PG,TRV,UNH,UTX,VZ,VZ,WMT,DIS
Percent invested in each stock,4.723962,0.0,18.850755,0.0,0.0,0.483583,0.0,0.0,0.0,0.734091,0.0,0.0,0.0,0.0,0.0,4.246867,0.0,4.015513,0.0,0.0,17.014174,0.0,10.700338,1.302144,37.928573,0.0,0.0,0.0,0.0,0.0


In [681]:
print '%.2f'%float(percentinF*100),'% invested in stocks' 
print '%.2f'%float(percentinrf*100),'% invested in risk-free assets' 
print '$%.2f total investment required' %float(req_funds)
print '$%.2f must be borrwed' %float(borrow)
print '%.2f'%float(((1+stdreturn)**12 - 1)*100),'% is the standard deviation (risk) of this investment'

45.44 % invested in stocks
54.56 % invested in risk-free assets
$1000000.00 total investment required
$0.00 must be borrwed
30.89 % is the standard deviation (risk) of this investment


Note that the fund $F$ has changed slightly to accommodate the change in the risk-free rate of return. Again, we are not required to borrow anything since we are not short selling. 45.4% of our funds should be invested in fund $F$, while the remaining 54.6% is to be placed in treasury notes. According to the Markowitz model, this is the portfolio makeup that will give us the best shot at achieving 10% annual returns over 10 years while minimizing our investment risk.

## Risk Metrics

Throughout this project, the ideas of standard deviation and risk have been invoked repeatedly, but it is not entirely intuitive how these numbers should be factored into our decision making. In this section, we will take a crack at quantifying risk in a few different ways and should emerge by the end with a deeper understanding of how risk may be assessed.

### Sharpe Ratio


The Sharpe ratio measures the trade-off between excess return and standard deviation. Intuitively, this ratio makes sense. If we want to avoid risk altogether, we are free to invest in a risk-free asset. However, the gains offered by doing this are lackluster, so we're willing to take on some extra risk by purchasing stocks (or maybe even shorting them) so that we may see some excess returns (i.e. returns greater than the risk-free rate). The Sharpe ratio quantifies this trade-off, and generally, the greater the Sharpe ratio, the more attractive the investment. The ratio is expressed as

$$\frac{\bar{r}_p - r_f}{\sigma_p}$$

where $\bar{r}_p$ is the average return of the portfolio, $r_f$ is the risk-free rate, and $\sigma_p$ is the standard deviation of the portfolio. Below is the calculation of the Sharpe ratio for our 10-year, restricted  fund $F$.

In [657]:
sharpe = ((np.mean(returns_F) - rf_10)/np.std(returns_F))
print 'The Sharpe Ratio is %.1f' %float(sharpe*100),'%'

The Sharpe Ratio is 31.2 %


The Sharpe ratio enjoys wide use within the financial world, but it breaks down when applied to non-normally-distributed portfolios. This is certainly something to look out for, and does somewhat limit its impact as a risk metric. For more on the Sharpe ratio and its applications and drawbacks, take a look at <a href="http://www.investopedia.com/terms/s/sharperatio.asp">this page<a>.

### VaR

Next, let's take a look at the Value-at-Risk (VaR) metric. VaR seeks to describe the potential for loss in terms of both the magnitude of loss and its likelihood. The method for calculating the VaR is rather convoluted, as it involves sorting historical portfolio returns to emulate a cumulative distribution function (CDF), choosing a confidence metric $p$, then approximating where the $p^{th}$ quantile would be if the data were continuous, rather than discrete. For this reason, we won't be going into the method in detail, but all of these steps are carried out in the code below.

In [661]:
#create CDF
weighted_returns = returnarray*w7
returns_F = np.sum(weighted_returns, axis = 1)#monthly mean portfolio return
cdf_x  = np.sort(returns_F)
cdf_y = np.array(range(int(months)))/months

p = 0.05 #confidence level
k = int(months*p) #approximate index of the p_th quantile
xk = cdf_x[k-1] #lower approximation of VaR
xk1 = cdf_x[k] #upper approximation of VaR
a = (months*p) - k #weighting factor
xp =  (1-a)*xk + a*xk1 #actual VaR
print 'The VaR is %.1f' %float(-1*xp*100),'%'

The VaR is 7.5 %


In [652]:
# Plot CDF
cdf = go.Scatter(
    x = cdf_x,
    y = cdf_y,
    name = 'CDF'
)

text = go.Scatter(
    x=[xp/2],
    y=[0.07],
    text=['VaR(X)'],
    mode='text',
    textfont=dict(
        size=18
    )
)


data = [cdf, text]

layout = dict(
    xaxis=dict(
        title='Portfolio Monthly Return'),
    yaxis=dict(
        title='Cumulative Distribution Function'), 
    )


layout.update(dict(
         shapes=[
        # Line Vertical
        {
            'type': 'line',
            'x0': xp,
            'y0': 0,
            'x1': xp,
            'y1': cdf_y[k],
            'line': {
                'color': 'rgb(255, 0, 0)',
                'width': 2,
            },
        },
        # Horizontal Vertical
        {
            'type': 'line',
            'x0': xp,
            'y0': cdf_y[k],
            'x1': 0,
            'y1': cdf_y[k],
            'line': {
                'color': 'rgb(255, 0, 0)',
                'width': 2,
            }
        }
        ]
    ))

fig = dict(data=data, layout=layout)
py.iplot(fig)

So this is telling us that the VaR, at a <i>credibility</i> level of $(1-p)=95\%$ is 7.5%. This means that in any given month, we have a 95% chance of losing less than 7.5% of our investment in fund $F$, and a 5% chance of losing more than 7.5%.

The VaR metric has its issues though, as it is not a <a href="https://en.wikipedia.org/wiki/Coherent_risk_measure">coherent risk measure </a> and can vastly understate the risk of events lying in the tails of portfolio return distributions. This can be dangerous, and so these problems have been addressed through the conditional value-at-risk (CVaR) method.

### CVaR

CVaR, unlike VaR, is indeed a coherent risk measure, and it seeks not to describe the general, overall risk of a portfolio, but to assess the likelihood of specific, large losses. The CVaR can be calculated by extracting the range of of returns equal to or worse than the VaR, and taking their weighted average. This is actually a rather simple task, but is slightly complicated again by the fact that we are trying to approximate a point in a hypothetical continuous distribution using only discrete data. 

In [667]:
#lower CVaR approximation
CVaR_k = np.zeros(int(months)) #initialize array of zeros
CVaR_k[0:k] = 1/(months*xk) #assign weights to sub-VaR returns
CVaR_k = sum(CVaR_k*cdf_x)

#upper CVaR approximation
CVaR_k1 = np.zeros(int(months)) #initialize arrat of zeros
CVaR_k1[0:k+1] = 1/(months*xk1) #assign weights to sub-VaR returns
CVaR_k1 = sum(CVaR_k1*cdf_x)

CVaR = (1-a)*CVaR_k + a*CVaR_k1 #actual CVaR

print 'The CVaR is %.2f' %float(CVaR*100),'%'

The CVaR is 7.02 %


This CVaR value tells us that, if and when we do see an extreme loss (a loss as bad or worse than out VaR), the expected value of this loss will be 7.02%. In other words, about 5% of the time, we can expect to lose 7.02% over the course of a month. This is a vital piece of information, because it tells us that when something bad does happen, it will hurt but it won't necessarily be catastrophic (although that assessment is subject to person risk tolerance).

## Conclusion

Over the course of this project, we have analyzed historical stock quotes, constructed Markowitz efficient frontiers using both analytical and numerical approaches, explored how the inclusion of short selling and risk-free assets affect our investment decisions, and assessed investment risk using a number of diverse methods. Along every step of the way, an approach to these problems using Python programming has been demonstrated. Hopefully, you have found the project easy to read and follow, and will feel comfortable building on these tools in your own studies.

On that note, it should be mentioned that the Markowitz model used here makes a number of assumptions that harm its validity in real-world applications. Perhaps chief among these is the assumption that while stock returns and variances are random variables, the covariance matrix is actually an unchanging feature of the universe. This is not the case, however, and the covariance between stocks in the past may have no bearing on their covariance in the future. 

The analysis performed in this project is still quite useful, as it has allowed us to gain a deeper understanding of portfolio theory and the application of mathematical modeling in finance. Much work and learning remains to be done, however, before one should feel comfortable deploying these tools in the actual market with actual funds. When that time finally comes, good luck!


.

### References

<ul>
<li> Leunberger (2013), Investment Science, 2nd Edition, Oxford University Press.
<li> van Dorp (2017), EMSE 6450 Class slides, url: <a href="http://www2.seas.gwu.edu/~dorpjr/EMSE292/Coursefiles.html">http://www2.seas.gwu.edu/~dorpjr/EMSE292/Coursefiles.html</a>
 </ul>

In [306]:
#py.init_notebook_mode(connected=True)
import plotly
import plotly.plotly as py
plotly.tools.set_credentials_file(username='wkuk101', api_key='ytxl38ufeHZMmiRNJgRF')

In [305]:
#Add custom CSS
from IPython.core.display import HTML
css_file = 'connor_CSS.css'
HTML(open(css_file, "r").read())